In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("flights").getOrCreate()
import json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
with open("../util/schema.json","r") as f:
    schema = StructType.fromJson(json.load(f))

22/12/27 15:03:35 WARN Utils: Your hostname, MacBook-Air-di-Teodoro.local resolves to a loopback address: 127.0.0.1; using 192.168.69.184 instead (on interface en0)
22/12/27 15:03:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/27 15:03:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("../data.nosync/cleaned/cleaned_flights.csv",schema=schema, header=True)

In [4]:
# get the different airports in Origin and Dest
airports_origin = df.select("Origin").distinct()
airports_dest = df.select("Dest").distinct()

# merge the two dataframes and remove duplicates
airports = airports_origin.union(airports_dest).distinct()

# convert to pandas dataframe
airports_pd = airports.toPandas()


In [5]:
# converto to a list
airports_list = airports_pd["Origin"].tolist()


In [7]:
'''
!wget https://www.ncei.noaa.gov/orders/cdo/3181703.csv
!wget https://www.ncei.noaa.gov/orders/cdo/3181706.csv
!wget https://www.ncei.noaa.gov/orders/cdo/3181708.csv
!wget https://www.ncei.noaa.gov/orders/cdo/3181724.csv
'''

--2022-12-27 17:09:27--  https://www.ncei.noaa.gov/orders/cdo/3181703.csv
Risoluzione di www.ncei.noaa.gov (www.ncei.noaa.gov)... 205.167.25.167, 205.167.25.177, 205.167.25.171, ...
Connessione a www.ncei.noaa.gov (www.ncei.noaa.gov)|205.167.25.167|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 10397206 (9,9M) [text/csv]
Salvataggio in: «3181703.csv»

3181703.csv         100%[===================>]   9,92M  3,49MB/s    in 2,8s    

2022-12-27 17:09:31 (3,49 MB/s) - «3181703.csv» salvato [10397206/10397206]

--2022-12-27 17:09:31--  https://www.ncei.noaa.gov/orders/cdo/3181706.csv
Risoluzione di www.ncei.noaa.gov (www.ncei.noaa.gov)... 205.167.25.167, 205.167.25.177, 205.167.25.171, ...
Connessione a www.ncei.noaa.gov (www.ncei.noaa.gov)|205.167.25.167|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 9733601 (9,3M) [text/csv]
Salvataggio in: «3181706.csv»

3181706.csv         100%[===================>]   9,28M  2,1

In [92]:
df_meteo = spark.read.csv("../data.nosync/meteo/",inferSchema=True, header=True)

In [79]:
df_correspondence = spark.read.csv("corrispondence.csv",inferSchema=True, header=True)

In [93]:
count = df_meteo.groupBy("STATION").agg({"*":"count"}).orderBy("count(1)")
# join df_meteo and count
df_meteo = df_meteo.join(count, on="STATION", how="inner")
# filter the stations with less than 300 value
df_meteo = df_meteo.filter(df_meteo["count(1)"] > 300)
# drop the count column
df_meteo = df_meteo.drop("count(1)")

df_meteo.groupBy("STATION").agg({"*":"count"}).orderBy("count(1)").show(100)


+-----------+--------+
|    STATION|count(1)|
+-----------+--------+
|USC00481840|     301|
|USW00003017|     304|
|USC00314471|     317|
|USW00025322|     335|
|USW00025329|     335|
|US1CABT0017|     336|
|USC00045280|     346|
|USW00004864|     347|
|RQW00011603|     357|
|USW00054831|     359|
|USW00003902|     359|
|USR0000AMOT|     360|
|US1FLOK0002|     362|
|USW00094938|     362|
|USW00012907|     364|
|USW00024132|     364|
|USW00093845|     365|
|USW00014737|     365|
|USW00014929|     365|
|USW00093821|     365|
|USW00012842|     365|
|USC00244558|     365|
|USW00094893|     365|
|USW00004781|     365|
|USW00026411|     365|
|USW00027502|     365|
|USW00023023|     365|
|USW00024131|     365|
|USW00013994|     365|
|USW00023169|     365|
|USW00014740|     365|
|USW00014836|     365|
|USW00024089|     365|
|USW00094746|     365|
|USW00024233|     365|
|USW00014850|     365|
|USW00094870|     365|
|USW00004803|     365|
|USW00024011|     365|
|USW00093822|     365|
|USW0000395

In [94]:
df_meteo_pd = df_meteo.toPandas()

22/12/27 18:08:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: STATION, NAME, LATITUDE, LONGITUDE, ELEVATION, DATE, AWND, AWND_ATTRIBUTES, DAPR, DAPR_ATTRIBUTES, EVAP, EVAP_ATTRIBUTES, FMTM, FMTM_ATTRIBUTES, MDPR, MDPR_ATTRIBUTES, PGTM, PGTM_ATTRIBUTES, PRCP, PRCP_ATTRIBUTES, PSUN, PSUN_ATTRIBUTES, SN32, SN32_ATTRIBUTES, SNOW, SNOW_ATTRIBUTES, SNWD, SNWD_ATTRIBUTES, SX32, SX32_ATTRIBUTES, TAVG, TAVG_ATTRIBUTES, TMAX, TMAX_ATTRIBUTES, TMIN, TMIN_ATTRIBUTES, TOBS, TOBS_ATTRIBUTES, TSUN, TSUN_ATTRIBUTES, WDF2, WDF2_ATTRIBUTES, WDF5, WDF5_ATTRIBUTES, WDMV, WDMV_ATTRIBUTES, WESD, WESD_ATTRIBUTES, WSF2, WSF2_ATTRIBUTES, WSF5, WSF5_ATTRIBUTES, WT01, WT01_ATTRIBUTES, WT02, WT02_ATTRIBUTES, WT03, WT03_ATTRIBUTES, WT04, WT04_ATTRIBUTES, WT05, WT05_ATTRIBUTES, WT06, WT06_ATTRIBUTES, WT07, WT07_ATTRIBUTES, WT08, WT08_ATTRIBUTES, WT09, WT09_ATTRIBUTES, WT10, WT10_ATTRIBUTES, WT11, WT11_ATTRIBUTES, WT13, WT13_ATTRIBUTES, WT14, WT14_ATTRIBUTES, WT15, WT15_ATTRIBUTES, WT1

In [95]:
# get the percentage of missing values for each column
df_meteo_pd.isnull().sum()/len(df_meteo_pd)

STATION            0.000000
NAME               0.000000
LATITUDE           0.000000
LONGITUDE          0.000000
ELEVATION          0.000000
DATE               0.000000
AWND               0.064842
AWND_ATTRIBUTES    0.064842
DAPR               0.999736
DAPR_ATTRIBUTES    0.999736
EVAP               0.993175
EVAP_ATTRIBUTES    0.993175
FMTM               0.993749
FMTM_ATTRIBUTES    0.993749
MDPR               0.975505
MDPR_ATTRIBUTES    0.975505
PGTM               0.575379
PGTM_ATTRIBUTES    0.575379
PRCP               0.383838
PRCP_ATTRIBUTES    0.383838
PSUN               0.728712
PSUN_ATTRIBUTES    0.728712
SN52               0.748075
SN52_ATTRIBUTES    0.748075
SNOW               0.379361
SNOW_ATTRIBUTES    0.379361
SNWD               0.240387
SNWD_ATTRIBUTES    0.240387
SX52               0.624989
SX52_ATTRIBUTES    0.624989
TAVG               0.704690
TAVG_ATTRIBUTES    0.704690
TMAX               0.391463
TMAX_ATTRIBUTES    0.391463
TMIN               0.043631
TMIN_ATTRIBUTES    0